In [ ]:
# ===========================
# Investigated the homogeneous binary nucleation of atmospherically-relevant molecules
# =========================== 

# 🧱 Generate Dataframe: 
#     Create fragmentation dataframe
# ---------------------------
# Table of Contents
# 1️⃣ Output each unary component coordinates into mol1 and mol2 directory
# 2️⃣ Perform cluster analysis on each mol1/mol2 of each frame
# 3️⃣ Write bash script to merge all tsv files in mol1/mol2 folder

In [1]:
# ===============================
# 1️⃣ Output each unary component coordinates into mol1 and mol2 directory
# ===============================
# 📆 Date created: October 12, 2024 
# 📆 Date updated: October 13, 2024 
# - this cell ouputs directory "output_files"

import pandas as pd
import os
import time
from tqdm import tqdm
import shutil

# Start timing the script
start_time = time.time()

# Read the Parquet file
df_combined = pd.read_parquet('nuc_fort-to-panda_combined.parquet')

# Define the output folder and subdirectories
output_folder = 'output_files'
mol1_folder = os.path.join(output_folder, 'mol1')
mol2_folder = os.path.join(output_folder, 'mol2')

# If the output folder exists, delete it and recreate it
if os.path.exists(output_folder):
    shutil.rmtree(output_folder)

os.makedirs(mol1_folder)
os.makedirs(mol2_folder)

# Define the priority list for molecule assignment
priority_list = ['SOL', 'NON', 'BUTA', 'MET']

# Get the unique frame numbers
frame_numbers = df_combined['frame_number'].unique()

print(f"Processing {len(frame_numbers)} frames...")

# Process each frame with a progress bar
for frame in tqdm(frame_numbers, desc="Frames processed"):
    df_frame = df_combined[df_combined['frame_number'] == frame]

    # Identify the largest cluster by counting the number of atoms per nucleus_id
    cluster_sizes = df_frame.groupby('nucleus_id').size()
    largest_cluster_id = cluster_sizes.idxmax()

    # Extract data for the largest cluster
    df_largest_cluster = df_frame[df_frame['nucleus_id'] == largest_cluster_id]

    # Get the unique molecule names present in the largest cluster
    molecule_names = df_largest_cluster['molecule_name'].unique()

    # Assign molecules to mol1 and mol2 based on the priority list
    molecules_present = [mol for mol in priority_list if mol in molecule_names]
    mol1_name = molecules_present[0] if len(molecules_present) > 0 else None
    mol2_name = molecules_present[1] if len(molecules_present) > 1 else None

    # Process mol1
    if mol1_name:
        df_mol1 = df_largest_cluster[df_largest_cluster['molecule_name'] == mol1_name].copy()
        df_mol1 = df_mol1[['molecule_name', 'x_coord', 'y_coord', 'z_coord']]
        df_mol1['molecule_name'] = df_mol1['molecule_name'].str[:3]
        mol1_file = os.path.join(mol1_folder, f'frame_{frame}.tsv')
        df_mol1.to_csv(mol1_file, sep='\t', header=False, index=False)

    # Process mol2
    if mol2_name:
        df_mol2 = df_largest_cluster[df_largest_cluster['molecule_name'] == mol2_name].copy()
        df_mol2 = df_mol2[['molecule_name', 'x_coord', 'y_coord', 'z_coord']]
        df_mol2['molecule_name'] = df_mol2['molecule_name'].str[:3]
        mol2_file = os.path.join(mol2_folder, f'frame_{frame}.tsv')
        df_mol2.to_csv(mol2_file, sep='\t', header=False, index=False)

# End timing the script
end_time = time.time()
elapsed_time = end_time - start_time

print(f"Processing completed in {elapsed_time:.2f} seconds.")

Processing 201 frames...


Frames processed: 100%|██████| 201/201 [00:24<00:00,  8.33it/s]

Processing completed in 26.02 seconds.


In [2]:
# ===============================
# 2️⃣ Write cluster analysis bash script to target mol1/mol2 of each frame
# ===============================
# 📆 Date created: October 12, 2024 
# 📆 Date updated: October 13, 2024 
# - this cell creates the bash script to employ cluster_nm.f on all structures in mol1 and mol2 folder

# ===============================
# Instructions
# ===============================
# [1] Create a directory "fragmentation_analysis"
#     mkdir fragmentation_analysis
# 
# [2] Transfer "output_files" generated previously and BATCH_ANALYSIS_EXAMINE.sh into "fragmentation_analysis"
#     mv output_files fragmentation_analysis
# 
# [3] Edit fort.10 file (do not transfer inside fragmentation_analysis folder) make sure that the following is in row 1
#     0 0                     ! NUMBER OF FRAMES
# 
# [4] Activate BATCH_ANALYSIS_EXAMINE.sh in "fragmentation_analysis"
#     ./BATCH_ANALYSIS_EXAMINE.sh

# Open the file in write mode
with open("BATCH_ANALYSIS_EXAMINE.sh", "w") as file:
    # Write the script line by line
    file.write("""#!/bin/bash
# PART2 OF ANALYSIS SCRIPTS
# DATE WRITTEN: AUGUST 27, 2020
# DATE UPDATED: OCTOBER 17, 2024
# PURPOSE: generate cluster reports
echo " "
echo " | COMMENCE: PART2 OF ANALYSIS SCRIPTS | GENERATE CLUSTER REPORTS | "
echo " "

# ========================================================
# ----------------    INSTRUCTIONS    --------------------
# ========================================================

# [1] Create a directory "fragmentation_analysis"
#     mkdir fragmentation_analysis
# 
# [2] Transfer "output_files" generated previously and BATCH_ANALYSIS_EXAMINE.sh into "fragmentation_analysis"
#     mv output_files fragmentation_analysis
# 
# [3] Edit fort.10 file (do not transfer inside fragmentation_analysis folder) make sure that the following is in row 1
#     0 0                     ! NUMBER OF FRAMES
# 
# [4] Activate BATCH_ANALYSIS_EXAMINE.sh in "fragmentation_analysis"
#     ./BATCH_ANALYSIS_EXAMINE.sh
#
# [5] Transfer "generate_fragmentation.sh" to mol1 and mol2
#     cp fragmentation.sh ./fragmentation_analysis/output_files/mol1
#     cp fragmentation.sh ./fragmentation_analysis/output_files/mol2
#
# [6] Activate "generate_fragmentation.sh"
#     ./generate_fragmentation.sh
#
# [7] Transfer output "fragmentation_{mol1_name}_fragmentation_{mol2_name}_{mol}.csv" 
#     to directory where 6_plot_fragmentation can read it

# ========================================================
# -------------------    SECTIONS    ---------------------
# ========================================================

# I.    DECLARE VARIABLES
#
# II.   SET SCRIPT FUNCTIONS
#
# III.  PROBE SIMULATION DATA

# <=======================================================
# <==== I. DECLARE VARIABLES
# <=======================================================

echo "...(1/3) DECLARE VARIABLES"

mol_dirs=("mol1" "mol2")
SECONDS=0  # Initialize timer

# <=======================================================
# <==== III. PROBE SIMULATION DATA
# <=======================================================

echo "...(3/3) PROBE SIMULATION DATA"

# Compile the cluster program
gfortran -O3 cluster_nm.f -o cluster || exit 1

# Check if fort.10 exists
if [[ ! -f fort.10 ]]; then
    echo "Error: fort.10 file not found."
    exit 1
fi

# Create a backup of fort.10
cp fort.10 fort.10.backup

# Loop over mol1 and mol2 directories
for mol_dir in "${mol_dirs[@]}"
do
    echo "Processing directory: $mol_dir"

    # Copy the cluster executable and fort.10 backup into the molecule directory
    cp cluster fort.10.backup output_files/"$mol_dir"/

    # Change to the molecule directory
    cd output_files/"$mol_dir" || exit 1

    # Loop over tsv files in the mol_dir
    for tsv_file in frame_*.tsv
    do
        echo "--------------------------------------------"
        echo "Processing file: $tsv_file"

        # Determine the molecule type from the first line of the tsv file
        molecule_name=$(head -n 1 "$tsv_file" | awk '{print $1}')

        # Truncate molecule_name to first 3 characters
        molecule_name=${molecule_name:0:3}

        # Map molecule_name to number of atoms per molecule
        case "$molecule_name" in
            SOL)
                atoms_per_molecule=4
                ;;
            NON)
                atoms_per_molecule=9
                ;;
            BUT)
                atoms_per_molecule=6
                ;;
            MET)
                atoms_per_molecule=3
                ;;
            *)
                echo "Error: Unknown molecule type: $molecule_name in file $tsv_file"
                # Restore original fort.10 before exiting
                cp fort.10.backup fort.10
                exit 1
                ;;
        esac

        # Calculate total number of atoms (lines) in the tsv file
        total_atoms=$(wc -l < "$tsv_file")

        # Calculate number of molecules
        number_of_molecules=$((total_atoms / atoms_per_molecule))

        echo "Molecule type: $molecule_name"
        echo "Total atoms: $total_atoms"
        echo "Atoms per molecule: $atoms_per_molecule"
        echo "Number of molecules: $number_of_molecules"

        # Get the frame number from the tsv file name
        frame_filename=$(basename "$tsv_file" .tsv)
        frame_number=${frame_filename#frame_}

        # Create a frame-specific folder
        frame_folder="frame_$frame_number"

        # Delete the frame folder if it exists
        if [[ -d "$frame_folder" ]]; then
            echo "Deleting existing folder: $frame_folder"
            rm -rf "$frame_folder"
        fi

        mkdir -p "$frame_folder"

        # Move the tsv file into the frame folder
        mv "$tsv_file" "$frame_folder/"

        # Copy the cluster executable and fort.10.backup into the frame folder
        cp cluster fort.10.backup "$frame_folder/"

        # Change to the frame folder
        cd "$frame_folder" || exit 1
        ls .

        # Prepare fort.10 file, updating the population line
        cp fort.10.backup fort.10

        # Replace the population line (line 5) in fort.10
        sed -i "6s/.*/$number_of_molecules                    ! POPULATION/" fort.10

        # Run cluster with the TSV file as input
        echo "Running cluster on TSV file: $tsv_file"
        ./cluster < "$tsv_file"
        cluster_exit_status=$?

        # Check if cluster exited with an error
        if [[ $cluster_exit_status -ne 0 ]]; then
            echo "Error: Cluster program failed on file $tsv_file with exit status $cluster_exit_status"
            echo "Terminating script."
            # Restore original fort.10
            cp fort.10.backup fort.10
            exit 1
        fi

        # Remove the cluster executable and fort.10.backup from the frame folder
        rm cluster fort.10 fort.10.backup

        # Move back to the molecule directory
        cd ..

    done

    # Remove the cluster executable and fort.10.backup from the molecule directory
    rm cluster fort.10.backup

    # Move back to the main directory
    cd ../..

done

# Restore original fort.10
mv fort.10.backup fort.10

# =========================================================
# ------------------------- END ---------------------------
# =========================================================

echo "...DONE"
echo " "

duration=$SECONDS
echo " | TIME ELAPSED: $(($duration / 60)) MINUTE/S and $(($duration % 60)) SECOND/S |"
echo " "
""")

In [5]:
# ===============================
# 3️⃣ Write bash script to merge all tsv files in mol1/mol2 folder
# ===============================
# 📆 Date created: October 12, 2024 
# 📆 Date updated: October 13, 2024 
# - this cell creates the bash script to employ cluster_nm.f on all structures in mol1 and mol2 folder

# ===============================
# Instructions
# ===============================
# [5] Transfer "generate_fragmentation.sh" to mol1 and mol2
#     cp fragmentation.sh ./fragmentation_analysis/output_files/mol1
#     cp fragmentation.sh ./fragmentation_analysis/output_files/mol2
#
# [6] Activate "generate_fragmentation.sh"
#     ./generate_fragmentation.sh
#
# [7] Transfer output "fragmentation_{mol1_name}_fragmentation_{mol2_name}_{mol}.csv" 
#     to directory where 6_plot_fragmentation can read it

# Define variables for mol1_name and mol2_name
mol1_name = "BUT"  # Replace with desired name for mol1
mol2_name = "MET"  # Replace with desired name for mol2
mol = "mol1" # or mol2

# Create the filename using the mol1_name and mol2_name variables
output_filename = f"fragmentation_{mol1_name}_fragmentation_{mol2_name}_{mol}.csv"

# Write the bash script to the file "generate_fragmentation.sh"
with open("generate_fragmentation.sh", "w") as file:
    file.write(f"""#!/bin/bash

# Initialize the output CSV file
output_file="{output_filename}"
> "$output_filename"  # Clear the file if it exists

# Add column headers at the beginning of the output file
echo "time,size,frequency" > "$output_filename"

# Iterate over each frame directory in numerical order
for frame_dir in $(ls -d frame_*/ | sort -V)
do
    # Extract the frame number from the directory name (e.g., "frame_0" -> "0")
    frame_number=${{frame_dir#frame_}}
    frame_number=${{frame_number%/}}
    
    # Calculate time as frame_number * 0.5
    time=$(echo "$frame_number * 0.5" | bc)

    # Check if fort.3100 exists in the current frame directory
    fort_file="${{frame_dir}}fort.3100"
    if [[ -f "$fort_file" ]]; then
        # Read each line in fort.3100, skipping header lines
        awk -v time="$time" 'NR > 1 {{print time "," $2 "," $3}}' "$fort_file" >> "$output_filename"
    else
        echo "Warning: $fort_file not found, skipping."
    fi
done

echo "Data compiled into $output_filename."
""")